In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [4]:
EPOCHS = 30
BATCH_SIZE = 64

transform = transforms.Compose([
  transforms.ToTensor()
])

trainset = datasets.FashionMNIST(
    root='./.data/',
    train=True,
    download=True,
    transform=transform
)

testset = datasets.FashionMNIST(
    root='./.data/',
    train=False,
    download=True,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=testset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 10)

  def forward(self, x):
    x = x.view(-1, 784)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [6]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [7]:
def train(model, train_loader, optimizer):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

In [8]:
def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(DEVICE), target.to(DEVICE)
      output = model(data)

      test_loss += F.cross_entropy(output, target, reduction='sum').item()

      pred = output.max(1, keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy

In [10]:
for epoch in range(1, EPOCHS+1):
  train(model, train_loader, optimizer)
  test_loss, test_accuracy = evaluate(model, test_loader)

  print('[{}] Test Loss : {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss : 0.6870, Accuracy: 75.04%
[2] Test Loss : 0.5934, Accuracy: 79.42%
[3] Test Loss : 0.5403, Accuracy: 81.13%
[4] Test Loss : 0.5152, Accuracy: 81.78%
[5] Test Loss : 0.5049, Accuracy: 82.03%
[6] Test Loss : 0.5048, Accuracy: 81.69%
[7] Test Loss : 0.4690, Accuracy: 83.58%
[8] Test Loss : 0.4853, Accuracy: 81.83%
[9] Test Loss : 0.4606, Accuracy: 83.47%
[10] Test Loss : 0.4464, Accuracy: 84.09%
[11] Test Loss : 0.4396, Accuracy: 84.19%
[12] Test Loss : 0.4389, Accuracy: 84.52%
[13] Test Loss : 0.4275, Accuracy: 84.97%
[14] Test Loss : 0.4254, Accuracy: 84.93%
[15] Test Loss : 0.4359, Accuracy: 84.62%
[16] Test Loss : 0.4368, Accuracy: 84.45%
[17] Test Loss : 0.4111, Accuracy: 85.37%
[18] Test Loss : 0.4065, Accuracy: 85.62%
[19] Test Loss : 0.3991, Accuracy: 85.99%
[20] Test Loss : 0.4008, Accuracy: 85.82%
[21] Test Loss : 0.4071, Accuracy: 85.51%
[22] Test Loss : 0.4014, Accuracy: 85.86%
[23] Test Loss : 0.3925, Accuracy: 86.18%
[24] Test Loss : 0.3829, Accuracy: 86.55%
[